# General debug

<font color='red'>**This notebook is outdated as of 9 May 2021.  
Please use more recent notebooks for help.**</font>

There have been a couple of inconsistencies and errors in the results and analysis of the LUVOIR small APLC PASTIS modes that I want to debug in here.

In [ ]:
# Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline
from astropy.io import fits
import astropy.units as u
import hcipy
from pastis.e2e_simulators.indexed_segmented_mirror import SegmentedMirror

from pastis.config import CONFIG_PASTIS
import pastis.util as util
from pastis.e2e_simulators.luvoir_imaging import LuvoirAPLC

In [ ]:
# Which directory are we working in?
savedpath = '/Users/ilaginja/Documents/data_from_repos/pastis_data/2019-8-09_001_1nm'    # medium apodizer LUVOIR
#savedpath = '/Users/ilaginja/Documents/data_from_repos/pastis_data/2019-8-11_001_1nm'    # large apodizer LUVOIR

#### Load the SM

In [ ]:
# Load aperture files needed for SM
nseg = 120
wvln = 638e-9

datadir = os.path.join(util.find_repo_location(), CONFIG_PASTIS.get('LUVOIR', 'optics_path_in_repo'))
aper_path = 'inputs/TelAp_LUVOIR_gap_pad01_bw_ovsamp04_N1000.fits'
aper_ind_path = 'inputs/TelAp_LUVOIR_gap_pad01_bw_ovsamp04_N1000_indexed.fits'
aper_read = hcipy.read_fits(os.path.join(datadir, aper_path))
aper_ind_read = hcipy.read_fits(os.path.join(datadir, aper_ind_path))

pupil_grid = hcipy.make_pupil_grid(dims=aper_ind_read.shape[0], diameter=15)
aper = hcipy.Field(aper_read.ravel(), pupil_grid)
aper_ind = hcipy.Field(aper_ind_read.ravel(), pupil_grid)

wf_aper = hcipy.Wavefront(aper, wvln)

# Load segment positions from fits header
hdr = fits.getheader(os.path.join(datadir, aper_ind_path))

poslist = []
for i in range(nseg):
    segname = 'SEG' + str(i+1)
    xin = hdr[segname + '_X']
    yin = hdr[segname + '_Y']
    poslist.append((xin, yin))
    
poslist = np.transpose(np.array(poslist))
seg_pos = hcipy.CartesianGrid(poslist)

# Instantiate SM
sm = SegmentedMirror(aper_ind, seg_pos)

#### Instantiate LUVOIR simulator

In [ ]:
# Instantiate LUVOIR
sampling = 4
apodizer_design = 'medium'
# This path is specific to the paths used in the LuvoirAPLC class
optics_input = os.path.join(util.find_repo_location(), CONFIG_PASTIS.get('LUVOIR', 'optics_path_in_repo'))

luvoir = LuvoirAPLC(optics_input, apodizer_design, sampling)

In [ ]:
# Make reference image and normalization factor
luvoir.flatten()
psf_unaber, ref = luvoir.calc_psf(ref=True)
norm = ref.max()

# Create DH
dh_outer = hcipy.circular_aperture(2*luvoir.apod_dict[apodizer_design]['owa'] * luvoir.lam_over_d)(luvoir.focal_det)
dh_inner = hcipy.circular_aperture(2*luvoir.apod_dict[apodizer_design]['iwa'] * luvoir.lam_over_d)(luvoir.focal_det)
dh_mask = (dh_outer - dh_inner).astype('bool')

# Calculate DH mean contrast
dh_intensity = psf_unaber/norm * dh_mask
baseline_contrast = np.mean(dh_intensity[np.where(dh_intensity != 0)])

# Display DH and mean contrast
plt.figure(figsize=(10, 5))
plt.subplot(121)
hcipy.imshow_field(ref/norm, norm=LogNorm(), mask=dh_mask)
plt.title('Reference PSF', size=15)
plt.subplot(122)
hcipy.imshow_field(psf_unaber/norm, norm=LogNorm(), mask=dh_mask)
plt.title('Unaberrated PSF', size=15)

print('Baseline contrast:', baseline_contrast)

## PASTIS matrix

In [ ]:
matrix = fits.getdata(os.path.join(savedpath, 'matrix_numerical', 'PASTISmatrix_num_piston_Noll1.fits'))

plt.figure(figsize=(10, 10))
plt.imshow(matrix)
plt.colorbar()

In [ ]:
print('Min matrix: {}'.format(np.min(matrix)))
print('Max matrix: {}'.format(np.max(matrix)))
print('A couple diagonal elements: {}'.format((matrix[0,0], matrix[10,10], matrix[15,15], matrix[50,50])))
print('A couple of off-axis elements: {}'.format((matrix[0,5], matrix[15,5], matrix[50,112])))

The PASTIS matrix is a symmetrical matrix.

In [ ]:
a = 5
b = 60

c = 13
d = 94

print(matrix[a,b], matrix[b,a])
print(matrix[c,d], matrix[d,c])

The (13, 94) pair is not exactly the same and it looks like this might stem from numerical errors. Let's try some more.

In [ ]:
e = 49
f = 43

g = 83
h = 119

print(matrix[e,f], matrix[f,e])
print(matrix[g,h], matrix[h,g])

I found another pair where it shows, (83, 119).

#### Matrix units

In the numerical PASTIS case, we fill the matrix with contrast values and then divide by the aberration squared. Now, in the LUVOIR case, while I define the aberration as 1nm in the configfile, when I apply it to the simulator that makes the images for the matrix, I convert that to meters by multiplying it with 1e-9. This is saved in the variable `nm_aber`.

**I carry `nm_aber` around in the matrix building script and only ever use it for the E2E simulator.** This means that when I get to the point of normalizing the matrix, `nm_aber` is still in meters. This is odd, since I don't actually apply the normalization factor when I build the matrix with 1nm as per the configfile.

I always assumed my matrix has untis of contrast over linear units squared, so $C/nm^2$. This seems to have been confirmed when I constructed the hockeystick contrast curve and I think this makes sense. In the hockeystick script, I define an rms range with respect to the input unit, meaning if my log range is fomr -4 to 4, then I am working with rms of 1e-4 nm to 1e4 nm. Also in that script, before applying the aberration to the E2E simulator, it gets converted to meters with astropy units.

## Mode basis and eigenvalues from SVD

The mode basis gets returned as `emodes[segmentnr, modesnr]`.

In [ ]:
u, s, vh = np.linalg.svd(matrix, full_matrices=True)

In [ ]:
evals = s
emodes = u

print(evals.shape)
print(emodes.shape)

In [ ]:
print('evals:\n{}'.format(evals))

In [ ]:
# Plot evals
plt.figure(figsize=(12, 7))
plt.plot(evals[:-1])
plt.title('Eigenvalues', size=15)
plt.xlabel('Modes', size=15)
plt.ylabel('Log Eigenvalues', size=15)
plt.semilogy()

The question is now, what units are the modes in?

In [ ]:
# Print one mode
modenr = 10
print('Mode {}:\n{}'.format(modenr, emodes[:, modenr]))

In [ ]:
aber_fac = 1e-9  # current fudgde factor on the modes, should be their units but not sure

In [ ]:
# Plot one mode
modenr = -2   # We start numbering at 0 here, 0-35 (Python nunmbering!)

sm.flatten()
for seg, val in enumerate(emodes[:, modenr]):
    #print(val)
    sm.set_segment(seg+1, aber_fac*val/2, 0, 0)

# Propagate WF and display SM phase
wf_sm = sm(wf_aper)

plt.figure(figsize=(10, 10))
hcipy.imshow_field(wf_sm.phase, cmap='RdBu')
plt.title(modenr+1)
plt.colorbar()

One thing to consider is that `imshow_field` displays the phase in units of radians here, wo it would be useful to figure out a way in which it actually displays it in OPD units.

One way of checking this is to put the modes onto the pupil one by one, propagate that and check the DH and its mean contrast. We know that all modes should have the same contrast contribution when applied to the pupil when they're multiplied by their maximum contribution value $\sigma_p$. So i fI calculate the $\sigma_p$ and then pick a mode $p$, multiply it by its $\sigma_p$ and apply that to the pupil and propagate, I sould get the same mean contrast for all the modes - whcih should be a 120th of the target contrast, which I will set to $C_t = 10^{-10}$.

## Maximum mode contributions

$$\sigma_p = \sqrt{\frac{C-C_0}{(N-1)\lambda_p}}$$

In [ ]:
# Calculate single sigma - remember that we start numbering at 0 because of python
def get_sigma(cstat, nseg, eigenval, c_zero):
    sigma = np.sqrt((cstat - c_zero) / ((nseg-1)*eigenval))
    return sigma

# Version without baseline contrast - which might be more accurate because I
# did not subtract C_0 when generating the matrix.
def get_sigma_no_c0(cstat, nseg, eigenval, c_zero):
    sigma = np.sqrt((cstat) / ((nseg-1)*eigenval))
    return sigma

In [ ]:
# Target contrast
c_target = 1e-10

In [ ]:
# Calculate the sigmas
#sigmas = get_sigma_no_c0(c_target, nseg, evals, baseline_contrast)
sigmas = get_sigma(c_target, nseg, evals, baseline_contrast)
print(sigmas.shape)
print(sigmas)

plt.figure(figsize=(8, 5))
plt.plot(sigmas[:-1])
plt.title('Maximum mode contribution', size=15)
plt.ylabel('$\sigma_p$', size=15)
plt.xlabel('Mode number', size=15)
plt.semilogy()

In [ ]:
# Load one mode on the pupil witih its sigma
modenr = -100   # We start numbering at 0 here, 0-35 (Python nunmbering!)

luvoir.flatten()
for seg, val in enumerate(emodes[:, modenr]):
    #print(val)
    luvoir.set_segment(seg+1, aber_fac*sigmas[modenr]*val/2, 0, 0)

# Calculate the LUVOIR PSF with that mode
psf_mode = luvoir.calc_psf(display_intermediate=True)

In [ ]:
# Plot just the DH PSF with one mode and its sigma
plt.figure(figsize=(8, 8))
hcipy.imshow_field(psf_mode, norm=LogNorm())
plt.title('DH from a single mode', size=15)
plt.colorbar()

# Get mean contrast from this DH
dh_mode = psf_mode/norm * dh_mask
mode_contrast = np.mean(dh_mode[np.where(dh_mode != 0)])
print('Mode contrast: {}'.format(mode_contrast))

In [ ]:
# Remind myself what the dict keys are called - only if calc_psf returns all planes (as 'intensity' or 'efield')
#print(all_planes.keys())

In [ ]:
# Apodizer plane
# plt.subplot(121)
# hcipy.imshow_field(all_planes['apod'].phase)
# plt.title('Phase')
# plt.subplot(122)
# hcipy.imshow_field(all_planes['apod'].intensity)
# plt.title('Intensity')

## Individual contrast contributions

Instead of doing a cumulative contrast plot that bunches all modes together, I can calculate the contrast contribution individually.

In [ ]:
aber_fac = 1e-9

# Individual contrast contributions
all_mode_psfs = []
all_mode_contrasts = []
all_mode_contrasts_squared_sigma = []

for onemode in range(len(evals)):
    print('{} of {}'.format(onemode+1, len(evals)))
    
    luvoir.flatten()
    for seg, val in enumerate(emodes[:, onemode]):
        #print(val)
        luvoir.set_segment(seg+1, aber_fac*sigmas[onemode]*val/2, 0, 0)

    # Calculate the LUVOIR PSF with that mode
    psf_mode = luvoir.calc_psf()
    all_mode_psfs.append(psf_mode)
    
    # Calc contrast
    dh_mode = psf_mode/norm * dh_mask
    mode_contrast = np.mean(dh_mode[np.where(dh_mode != 0)])
    all_mode_contrasts.append(mode_contrast)

If I plot the contrasts coming from the individual modes, I should get a completely straight line at $C_t/120$

In [ ]:
print('C_t / nseg = C_p')
print('{} / {} = {}'.format(c_target, nseg, c_target/nseg))
print('aberration fudge factor: {}'.format(aber_fac))

plt.figure(figsize=(16, 8))
plt.title('Target contrast ' + str(c_target))
plt.plot(all_mode_contrasts)
plt.semilogy()
#plt.ylim(4e-11, 4.5e-11)

## Single mode with increasing rms

Check the dependency of contrast on mode (phase) amplitude

#### Pure modes, no sigmas

In [ ]:
aber_fac = 1e-9

rms_list = np.linspace(0.01, 5, 50)
onemode_list = [1, 2, 50, 51]#, 52, 100, 101, 118, 119]
contrast_by_aberampl = np.zeros((len(onemode_list), len(rms_list)))
nm_aber_list = np.zeros((len(onemode_list), len(rms_list)))

for m, onemode in enumerate(onemode_list):
    print('Mode {} of {}'.format(m, len(onemode_list)))

    for j, rms in enumerate(rms_list):
        # Create random aberration coefficients
        aber = np.random.random([nseg])   # piston values in input units
        #print('PISTON ABERRATIONS: ', aber)

        # Normalize to the RMS value I want
        rms_init = util.rms(aber)
        aber *= rms / rms_init
        calc_rms = util.rms(aber)
        print("Calculated RMS: {} nm".format(calc_rms))
        nm_aber_list[m,j] = calc_rms

        # Remove global piston
        aber -= np.mean(aber)

        luvoir.flatten()
        for seg, (val, ab) in enumerate(zip(emodes[:, onemode], aber)):
            #print(val)
            luvoir.set_segment(seg+1, aber_fac*ab*val/2, 0, 0)

        # Calculate the LUVOIR PSF with that mode
        psf_mode = luvoir.calc_psf()
        all_mode_psfs.append(psf_mode)

        # Calc contrast
        dh_mode = psf_mode/norm * dh_mask
        mode_contrast = np.mean(dh_mode[np.where(dh_mode != 0)])
        contrast_by_aberampl[m,j] = mode_contrast

        #print(mode_contrast)

In [ ]:
plt.figure(figsize=(15, 10))
for i, mo in enumerate(onemode_list):
    plt.plot(nm_aber_list[i], contrast_by_aberampl[i], label='Mode '+str(mo))
plt.ylabel('Mean contrast', size=15)
plt.xlabel('Phase amplitude (nm)', size=15)
plt.title('No sigmas', size=15)
plt.legend()
plt.semilogy()

print('aber_fac = {}'.format(aber_fac))

In [ ]:
mode = 1
plt.figure(figsize=(15, 10))
for i, mo in enumerate(onemode_list):
    plt.plot(nm_aber_list[i], contrast_by_aberampl[i], label='Mode '+str(mo))
plt.ylabel('Mean contrast', size=15)
plt.xlabel('Phase amplitude (nm)', size=15)
plt.title('No sigmas', size=15)
plt.xlim(0, 1.)
plt.legend()
plt.semilogy()

print('aber_fac = {}'.format(aber_fac))

#### Including the sigmas

In [ ]:
#aber_fac = 1e-9

rms_list = np.linspace(0.01, 5, 50)
onemode_list = [1, 2, 50, 51, 52, 100, 101, 118, 119]
contrast_by_aberampl = np.zeros((len(onemode_list), len(rms_list)))
nm_aber_list = np.zeros((len(onemode_list), len(rms_list)))

for m, onemode in enumerate(onemode_list):
    print('Mode {} of {}'.format(m, len(onemode_list)))

    for j, rms in enumerate(rms_list):
        # Create random aberration coefficients
        aber = np.random.random([nseg])   # piston values in input units
        #print('PISTON ABERRATIONS: ', aber)

        # Normalize to the RMS value I want
        rms_init = util.rms(aber)
        aber *= rms / rms_init
        calc_rms = util.rms(aber)
        print("Calculated RMS: {} nm".format(calc_rms))
        nm_aber_list[m,j] = calc_rms

        # Remove global piston
        aber -= np.mean(aber)

        luvoir.flatten()
        for seg, (val, ab) in enumerate(zip(emodes[:, onemode], aber)):
            #print(val)
            luvoir.set_segment(seg+1, sigmas[onemode]*aber_fac*ab*val/2, 0, 0)

        # Calculate the LUVOIR PSF with that mode
        psf_mode = luvoir.calc_psf()
        all_mode_psfs.append(psf_mode)

        # Calc contrast
        dh_mode = psf_mode/norm * dh_mask
        mode_contrast = np.mean(dh_mode[np.where(dh_mode != 0)])
        contrast_by_aberampl[m,j] = mode_contrast

        #print(mode_contrast)

In [ ]:
plt.figure(figsize=(15, 10))
for i, mo in enumerate(onemode_list):
    plt.plot(nm_aber_list[i], contrast_by_aberampl[i], label='Mode '+str(mo))
plt.ylabel('Mean contrast', size=15)
plt.xlabel('Phase amplitude (nm)', size=15)
plt.title('With sigmas', size=15)
plt.legend()
plt.semilogy()

print('aber_fac = {}'.format(aber_fac))

In [ ]:
mode = 1
plt.figure(figsize=(15, 10))
for i, mo in enumerate(onemode_list):
    plt.plot(nm_aber_list[i], contrast_by_aberampl[i], label='Mode '+str(mo))
plt.ylabel('Mean contrast', size=15)
plt.xlabel('Phase amplitude (nm)', size=15)
plt.title('With sigmas', size=15)
plt.xlim(0, 2.)
plt.legend()
plt.semilogy()

print('aber_fac = {}'.format(aber_fac))